# Data Preprocessing and Exploratory Data Analysis

## Data Preprocessing
Now that we've scrapped all the data, we can start looking into it prior to performing some sort of analysis.
To make things a bit more interesting, let's load the `data_large.csv` file. This file is very similar to the output from the `scraping` notebook, but it was performed over a distance of 9999 (the US is ~3000 miles across).
We can easilly load CSV files into a `pandas.DataFrame` with the `read_csv` method!

In [1]:
import pandas as pd

In [25]:
df_large = pd.read_csv("./../assets/data_large.csv")

Now that the dataframe is loaded, let's check out the size (rows, columns) of the data. This is done by checking the `shape` attribute. The returned output is a tuple identifying the number of rows and columns.

In [13]:
print(df_large.shape)

(38912, 5)


38,912 entries with 5 features! Not too shabby. Let's see what a few of the entries look like. There are a few ways to do this and the common methods to use are `head()`, `tail()`, and `sample()`. Each one's first argument is the number of entries to return, by default the first two methods return 5 entires, and sample returns a single one.
* Personally, I prefer using `sample()` for most situations. The former two are great for time series data however, or anything that has an order to it.

In [20]:
df_large.sample(8)  # recall that the 8 means number of entries to return

,Unnamed: 0,city,searched_zipcode,url,content
10167,10167,NewYork,10025,https://newyork.craigslist.org/mnh/res/d/exp-h...,Experienced House Mgr. Available\nI've worked ...
20974,20974,Chicago,60618,https://grandrapids.craigslist.org/res/d/exter...,Experienced Siders\n\nExperienced roofers\n\nE...
20246,20246,Chicago,60618,https://chicago.craigslist.org/nwc/res/d/offic...,Experience cleaning lady is available for offi...
11505,11505,Dallas,75217,https://oklahomacity.craigslist.org/res/d/no-j...,"Power washing,yard cleaning, and any similar s..."
31793,31793,NewYork,10029,https://newyork.craigslist.org/mnh/res/d/itali...,"Dear Business Owners & Financial Investors, I ..."
30736,30736,Charlotte,28269,https://charlotte.craigslist.org/res/d/indepen...,"Hello, my name is Bruce, I'm a retired transpo..."
16870,16870,NewYork,10002,https://hudsonvalley.craigslist.org/res/d/need...,Can't afford to hire a bookkeeper full time or...
24672,24672,Sacramento,95823,https://sfbay.craigslist.org/sby/res/d/class-e...,"I have a Class A for 18 years, I have no point..."


Looks familiar! But what is the `Unnamed: 0` column? By default, when we export the `DataFrame` as a CSV, it saves the index number as a new column, and when it is reloaded, it does not assume that the first column is the index. There are a few ways to go about this:
* When exporting with `DataFrame.to_csv()` use the argument `index=False` to prevent the index column being saved.
* When importing with `pandas.read_csv()` use the argument `index_col=0` to specify that the first (or other column) is the index.
* Drop the column after importing the CSV.

Since the data is already loaded, let's use the last method of dropping the column.

In [26]:
df_large.drop(['Unnamed: 0'], axis=1, inplace=True)  # inplace means that it modifys the variable directly, equivilent to df_large = df_large.drop(...)

df_large.sample()

,city,searched_zipcode,url,content
6776,LosAngeles,90044,https://orangecounty.craigslist.org/res/d/prog...,CORE COMPETENCIES\n\nCxO-Level Business Strate...


Great! Now, another way to slim down the dataset is to check for duplicates. Remember how this larger set has its search distance at effectively the entire US? Well, as a result, there's a pretty good chance we're going to get duplicate entries. There's multiple ways to check for this condition:
* Use the `DataFrame.duplicated()` method, which returns a boolean series of which rows and duplicates.
* Use the `DataFrame.nunique()` method to figure out how many unique values there are. 

The issue with the first method is that it check if entries are identical. Since duplicates may exist from different zipcode searches, it may not return the duplicate's we're interested in (content or url). A simple way to check the number of duplicated values found with this approach is to `sum()` the result. This works because the return value of `duplicated()` is a boolean value per row, where False = 0 and True = 1. 

Let's run both methods!

In [37]:
print('Sum of duplicated: ', df_large.duplicated().sum())
print('\nnunique results:\n', df_large.nunique())

Sum of duplicated:  20

nunique results:
 city                   18
searched_zipcode       29
url                 13967
content             12933
dtype: int64


Ah! As we expected, duplicated perform what we'd like. The function does accept arguments however that will allow you to perform what we're looking for (You can pass a list of columns to check, i.e. `df_large.duplicated(['content'])` which will return a boolean series). 

Nonetheless however!  We find that `nunique()` did what we needed, we find that there are 12,933 unique resume's. Let's actually use the `duplicated()` method here to keep just the unique resumes. 

In [43]:
df_large = df_large[~df_large.duplicated(['content'])]

print("New dataframe size: ", df_large.shape)

New dataframe size:  (12933, 4)


First, we find that the new dataframe size matches the number of unique values found in the cell above. 

The line above may look a little confusing, so let's break it down.

1. `df_large.duplicated(['content'])` finds all the duplicated rows and by default, only keeps the first instance of it. This results in a boolean series, which denotes True/False for each entry.
2. The tilde (\~) denotes negation, which simply means flip all True/False values. This in effect represent's all the entries that are not duplicated. 
3. `df_large[~df_large.duplicated(['content'])]` is called indexing, which says return `df_large` where entires in the brackets are True, in this case the not duplicated rows. 
4. We reassign df_large to this new "not duplicated" version. 

Sweet! Now the data is a little cleaner, we've reduced the size to under a third of the original!

Some other useful sanity checks include: 
* `DataFrame.describe()` which displays measures of central tendency and some other metrics on all applicable columns (numerical).
* `DataFrame.dtypes` is an attribute that keeps track of all the data types used. Useful for larger sets.

Some of you may have noticed that the URL link may not match the `City` value. This may result in issues later on. A simple fix to this is to use the url city as opposed to the only found during the search, especially since we dropped duplicates on the content feature whilst retaining only the first encountered entry.  We'll fix this with the line below.

In [63]:
sr_split = df_large['url'].str.split("//", expand=True)[1]
sr_split = df_split.str.split(".", expand=True)[0]
df_large['city'] = sr_split

df_large.sample(5)

,city,searched_zipcode,url,content
37915,phoenix,85364,https://phoenix.craigslist.org/evl/res/d/handy...,Anyone looking for help around the house or mo...
30132,raleigh,28269,https://raleigh.craigslist.org/res/d/affordabl...,I've been a personal private duty in-home care...
30268,raleigh,28269,https://raleigh.craigslist.org/res/d/looking-f...,Hey my name is Shy 24 years old and I’m hoping...
8286,hudsonvalley,10025,https://hudsonvalley.craigslist.org/res/d/seek...,28 year old male in need of employment before ...
21585,nashville,37211,https://nashville.craigslist.org/res/d/moving-...,"Offering small moves, deliveries, packing , lo..."


Woah, what the heck? Sorry, the above is not very Pythonic, but it is more efficient than looping through each row. Let's break it down.

1. `pandas.Series.str.split()` acts similarly to the `split()` method on strings but is performed on an entire `pandas.Series` instead; it breaks the up the strings by the first argument. The `expand` argument makes each split a new column as opposed to a single column with a list, which is the default. 
2. Since the returned value from the split function is a `pandas.DataFrame`, the column names represent the split index (again, similar to a string split). When we split by `//`, we're removing the `https://` portion of the string, and taking the portion of the string after it by taking the 1st index.
3. We do the same thing, but this time split at the period, and return the first split value, which is the city via url.
4. Since we're taking a single column from the `pandas.DataFrame`, the variable will default to a series, which we can use to replace the `city` column in our original `DataFrame`.

Let's see how many cities we now have!

In [65]:
df_large.nunique()

city                  193
searched_zipcode       26
url                 12933
content             12933
dtype: int64

Makes a lot more sense than the 18 we previously had! 

At this point, the `searched_zipcode` is probably useless and may actually be a bit confusing since it may not match up to the city. Let's drop it and continue our analysis.

In [66]:
df_large.drop(['searched_zipcode'], axis=1, inplace=True)

Alright! Enough of this cleaning stuff, let's get to some analysis!

## Analysis

There are a _ton_ of visulation packages available in Python (though they're not a pretty and powerful as some R ones), but by far the most commonly used is matplotlib, which is maintained by NumFOCUS. We'll be exploring some plots in both matplotlib and a higher level version based off of it called seaborn (easier to make pretty plots). 